# Taller 1
_____

<div style="text-align: justify"> Para el desarrollo del presente taller se utilizó una serie de funciones auxiliares con el objetivo de facilitar la construcción de los modelos matemáticos requeridos para el cálculo de ajustes, errores e indicadores. Dichas funciones son explicadas con mayor detalle a continuación.</div> 

In [2]:
import os
import pandas as pd
import scipy as sp
from scipy import stats
%pylab inline

Populating the interactive namespace from numpy and matplotlib


<div style="text-align: justify">La primer función auxiliar utilizada fue denominada **_importdata_**. Esta recibe como parámetro el nombre de la ciudad de donde se quieren importar los datos de radiación y retorna un arreglo de dos posiciones en el cual se encuentran los datos históricos del IDEAM y del NREL respectivamente. </div><br/>

<div style="text-align: justify"> Los datos están disponibles para las siguientes ciudades (escritas tal como aparecen en los directorios):</div>

* Barranquilla
* Cali
* Carmendebolivar
* Medellin
* SantaMarta
* Sucre
* Uribia
* Valledupar

In [6]:
def importdata(ciudad):
    
    base_dir = os.path.join( '.','data')
    fuente1 = 'IDEAM'
    fuente2 = 'NREL'
    serie_csv1 = os.path.join(base_dir,ciudad,fuente1+'.csv')
    serie_pd1 = pd.read_csv(serie_csv1, sep = ';',
                    encoding='latin-1')
    serie_csv2 = os.path.join(base_dir,ciudad,fuente2+'.csv')
    serie_pd2 = pd.read_csv(serie_csv2, sep = ';',
                    encoding='latin-1')
    
    return serie_pd1, serie_pd2
    

<div style="text-align: justify">La segunda función auxiliar utilizada fue denominada **_datapiece_**. Esta recibe como parámetros una lista de datos y dos fechas en la forma: _año inicial, año final, mes inicial, mes final, día inicial y día final_. Con base en esto, la función retorna la misma lista de datos ingresada pero recortada entre el intervalo de tiempo indicado. </div>

In [8]:
def datapiece(data, year1, year2, month1, month2, day1, day2):
    piece=data[(data['AÑO']>=year1) & (data['AÑO']<=year2)
            & (data['MES']>=month1) & (data['MES']<=month2)
            & (data['DIA']>=day1) & (data['DIA']<=day2)]
    return piece

<div style="text-align: justify">La tercera función auxiliar utilizada fue denominada **_datadj_**. Esta recibe como parámetros dos listas de datos y tiene como propósito ajustar el tamaño y temporalidad de la primera con respecto a la segunda. Esto permite que las listas puedan ser comparables entre si ya que la función retorna la primera lista ajustada. </div>

In [ ]:
#La variable de entrada data2 es IDEAM
def datadj(data1,data2):
    data1_adj=data1[(data1['AÑO']>=data2.iloc[0,0]) & (data1['AÑO']<=data2.iloc[-1,0])
            & (data1['MES']>=data2.iloc[0,1]) & (data1['MES']<=data2.iloc[-1,1])
            & (data1['DIA']>=data2.iloc[0,2]) & (data1['DIA']<=data2.iloc[-1,2])]
    
    return data1_adj


In [ ]:
def ghi_monthavg(serie_horaria):
    agnos = sorted(serie_horaria['AÑO'].unique())
    ghi_df = pd.DataFrame(columns=["AÑO", "MES", "GHIdiario"])
    for agno in agnos:
        for mes in range(1,13):
            nombre_energia = list(serie_horaria)[-1]
            dias = sorted(serie_horaria[(serie_horaria['AÑO'] == agno) &
                        (serie_horaria['MES'] == mes)]["DIA"].unique())
            ghi_dia = serie_horaria.loc[((serie_horaria["AÑO"] == agno)
                            & (serie_horaria["MES"] == mes)),
                                        nombre_energia].sum()/(dias[-1]*1000)
            ghi_dict = {"AÑO":agno, "MES":mes,
                        "GHIdiario":ghi_dia}
            ghi_df = ghi_df.append(ghi_dict, ignore_index=True)
    return ghi_df

In [ ]:
def ENFICC(data1):
    
    ghi_diario_loc = ghi_monthavg(data1)
    percentil = 0.01
    enficc = ghi_diario_loc.quantile(q=percentil, 
                                 numeric_only=True)["GHIdiario"]

    return print('La ENFICC 99% es: ', enficc)

In [ ]:
def graphED(arrayIDEAM,arrayNRELMOD,arrayNREL):
    
    NRELMODadj=datadj(arrayNRELMOD,arrayIDEAM)
    NRELadj=datadj(arrayNREL,arrayIDEAM)

    ghi_diario_loc1 = ghi_monthavg(arrayIDEAM)    
    datos_IDEAM1 = sorted(ghi_diario_loc["GHIdiario"])
    numdatos1 = range(len(datos_IDEAM1))

    ghi_diario_loc2 = ghi_monthavg(NRELMODadj)    
    datos_IDEAM2 = sorted(ghi_diario_loc2["GHIdiario"])
    numdatos2 = range(len(datos_IDEAM2))

    ghi_diario_loc3 = ghi_monthavg(NRELadj)    
    datos_IDEAM3 = sorted(ghi_diario_loc3["GHIdiario"])
    numdatos3 = range(len(datos_IDEAM3))

    pylab.xlabel("Datos")
    pylab.ylabel("Energía/Energía Base IDEAM")
    
    pylab.plot(numdatos1,datos_IDEAM1/datos_IDEAM1[0],'-k', label='IDEAM')
    pylab.plot(numdatos2,datos_IDEAM2/datos_IDEAM1[0],'-b', label='NRELMOD')
    pylab.plot(numdatos3,datos_IDEAM3/datos_IDEAM1[0],'-r', label='NREL')
        
    pylab.legend(loc='upper left')
    output_dir = os.path.join( '.','output')
    pylab.savefig(os.path.join(output_dir,'valledupar_IDEAM.png'), dpi=600)
    
    return pylab.plot()

In [ ]:
def MCP(data_ref,data_obj):
    
    #Extrae los datos de referenia en el intervalo de tiempo de los datos objetivo 
    data_ref_adj=datadj(data_ref,data_obj)
    #Calcula los estadísticos de los datos en el periodo común
    mu_obj=data_obj['IDEAM'].mean()
    sigma_obj=data_obj['IDEAM'].std()
    mu_ref=data_ref_adj['NREL'].mean()
    sigma_ref=data_ref_adj['NREL'].std()
    
    #Calcula los parámetros del modelo
    m=sigma_obj/sigma_ref
    b=mu_obj-m*mu_ref
    
    #Aplica el modelo remplazando los valores negativos con 0
    data_ref_cor=data_ref['NREL']*m+b
    data_ref_cor[data_ref_cor < 0] = 0
    
    dat2 = pd.DataFrame({'NRELMOD': data_ref_cor})
    newdata=data_ref.join(dat2['NRELMOD'])
    
               
    return newdata

In [ ]:
def RMSEn(data1,column1,data2,column2):
    
    #Ajusta la primera lista de datos para que concuerde con el periodo de la segunda lista
    data1_adj=datadj(data1,data2)
    
    #Aplica la función de error cuadratico medio normalizado paso a paso
    fst=(data1_adj[column1]-data2[column2])**2
    N=len(data2.index)
    scnd=sqrt(fst.sum()/N)
    x_d2_max=data2[column2].max()
    x_d2_min=data2[column2].min()
    rmsen=scnd/(x_d2_max-x_d2_min)
    
    return rmsen

In [ ]:
def MBE(data1,column1,data2,column2):
    
    #Ajusta la primera lista de datos para que concuerde con el periodo de la segunda lista
    data1_adj=datadj(data1,data2)
    
    #Aplica la función de error de sesgo medio paso a paso
    fst=(data1_adj[column1]-data2[column2])
    scnd=data2[column2].sum()
    mbe=fst.sum()/scnd
    
    return mbe

In [ ]:
def KSI(data1,column1,data2,column2):
    
    data1_adj=datadj(data1,data2)
    
    ksi=sp.stats.ks_2samp(data1_adj[column1],data2[column2])
    return ksi[1]

<div style="text-align: justify"> </div>

<div style="text-align: justify"> </div>